
#### 1.1 How many goals were attempted by head
#### 1.2 How many matches had attendees more than 12000
#### 2.1 Shots list where goal was scored outside 16m box sorted by descending order
#### 2.2 Rank teams based on average attendance at home games 

In [1]:
from MySqlStruct import *

@measure_execution_time
def MySqlExecTime():
    return MySql_connect()

print('----Loading MySql----')
result, execution_time = MySqlExecTime()  # Get both the result and execution time

# Unpack the values from the result
cursor,connection,df_shots,df_matches,df_players,df_teams = result

----Loading MySql----
------- Index(['Player', 'Outcome', 'Distance', 'Body Part', 'match_id'], dtype='object')
Inserted 245180 rows into shots.
------- Index(['Date', 'league', 'Round', 'Day', 'Attendance', 'home_id', 'away_id',
       'score_away', 'score_home', 'position_home', 'position_away',
       'home_capitan', 'away_capitan', 'home_xg', 'away_xg', 'home_xga',
       'away_xga', 'home_formation', 'away_formation', 'Timestamp'],
      dtype='object')
Inserted 14148 rows into matches.
------- Index(['id', 'name', 'Pos', 'Matches', 'club_id', 'MP', 'year'], dtype='object')
Inserted 20414 rows into all_players.
------- Index(['id', 'name', 'league', 'W', 'D', 'L', 'MP', 'GF', 'GA', 'GD', 'Pts',
       'xG', 'xGA', 'xGD', 'xGD/90'],
      dtype='object')
Inserted 675 rows into teams.

Data inserted successfully for all files!
Execution time: 9.20624828338623 seconds


In [2]:
# print(df_shots.columns)
# print(df_shots.head(5))
# print(df_matches.columns)
# print(df_matches.head(5))
# print(df_players.columns)
# print(df_players.head(5))
# print(df_teams.columns)
# print(df_teams.head(5))


# 1.1 How many goals were attempted by head

In [3]:
@measure_execution_time
def MySQLQ1_1():
    # Define the query
    query = "SELECT * FROM shots WHERE Outcome = 'Goal' AND `Body Part` = 'Head';"
    
    # Execute the query
    cursor.execute(query)
    
    # Fetch all the results
    result = cursor.fetchall()
    
    # Print the first result
    if result:
        print(result[0])
    
    # Print the number of records
    print(f"Number of rows in result: {len(result)}")
    print('Query 1.1')

# Assuming you have the db_connection already defined
result, execution_time = MySQLQ1_1()

(1, '4a1a9578', 'Goal', 8.0, 'Head', 'de515487')
Number of rows in result: 3963
Query 1.1
Execution time: 0.12302327156066895 seconds


# 1.2 How many matches had attendees more than 12000

In [4]:
@measure_execution_time
def MySQLQ1_2():
    # Define the query
    query = "SELECT * FROM matches WHERE Attendance > 12000;"

    # Execute the query
    cursor.execute(query)

    # Fetch all the results
    result = cursor.fetchall()

    # Print the first result (if any)
    if result:
        print(result[0])

    # Print the number of rows in the result
    print(f"Number of rows in result: {len(result)}")

    print('Query 1.2')

# Call the function
result, execution_time = MySQLQ1_2()


(2, '2022-10-09', 11, 11, 'Sun', 24671, '04eea015', '922493f3', 2, 2, 59, 41, '/en/players/75f2c59f/Roberto-Pereyra', '/en/players/2a1beb34/Marten-de-Roon', 1.0, 2.1, 2.1, 1.0, '3-5-2', '3-4-1-2', '2022-10-09 15:00:00')
Number of rows in result: 9166
Query 1.2
Execution time: 0.06301236152648926 seconds


#### 2.1 Total Goals Scoared and Inside VS Outside 16m box

#### 2.2 Rank teams based on average attendance at home games

In [5]:
@measure_execution_time
def MySQLQ2_2(cursor):
    # Query to get average attendance and count of matches by home_id
    query = """
    SELECT 
        m.home_id,
        AVG(m.Attendance) AS average_attendance,
        COUNT(*) AS matches_count,
        t.name AS team_name
    FROM matches m
    INNER JOIN teams t ON m.home_id = t.id
    GROUP BY m.home_id, t.name
    ORDER BY average_attendance DESC
    """
    
    cursor.execute(query)
    results_from_matches = cursor.fetchall()  # List of tuples
    
    # Convert the result to a list of dictionaries for easier handling in Python
    results_from_matches = [
        {
            'home_id': row[0],
            'average_attendance': row[1],
            'matches_count': row[2],
            'team_name': row[3]
        } 
        for row in results_from_matches
    ]
    
    # Convert to DataFrame for easy viewing and sorting
    df = pd.DataFrame(results_from_matches)
    df = df[['team_name', 'average_attendance', 'matches_count']]
    print(df.head(10))  # Display the top 10 rows

    # return results_from_matches

# Call the function and measure execution time
result, execution_time = MySQLQ2_2(cursor)


           team_name average_attendance  matches_count
0  Manchester United         56407.2273            176
1           Dortmund         56036.0216            139
2            Benfica         54471.1429             14
3       Hamburger SV         52331.0000              3
4          Barcelona         51691.4586            157
5      Bayern Munich         50702.6525            141
6        Real Madrid         47265.5404            161
7    West Ham United         46952.0690            145
8            Arsenal         46577.0500            160
9     Internazionale         46508.7736            159
Execution time: 0.035003662109375 seconds


In [ ]:
@measure_execution_time
def MySQLQ2_1():
    # SQL query to filter only goals (Outcome = 'Goal') and calculate total goals per player
    query = """
    SELECT 
        SUM(CASE WHEN s.distance > 16 THEN 1 ELSE 0 END) AS goals_gt_16,  -- Sum the goals where distance > 16
        SUM(CASE WHEN s.distance <= 16 THEN 1 ELSE 0 END) AS goals_lt_16,  -- Sum the goals where distance <= 16
        COUNT(*) AS total_goals  -- Total goals (count all records where Outcome = 'Goal')
    FROM shots s
    WHERE s.Outcome = 'Goal'  -- Only include records where Outcome = 'Goal'
    GROUP BY s.Player  -- Group by player name
    ORDER BY total_goals DESC
    """

    query2 = '''
    SELECT 
        ap.name AS player_name,  -- Select the player name from the all_players table
        SUM(CASE WHEN s.distance > 16 THEN 1 ELSE 0 END) AS goals_gt_16,  -- Sum the goals where distance > 16
        SUM(CASE WHEN s.distance <= 16 THEN 1 ELSE 0 END) AS goals_lt_16,  -- Sum the goals where distance <= 16
        COUNT(*) AS total_goals  -- Total goals (count all records where Outcome = 'Goal')
    FROM shots s
    JOIN all_players ap ON s.Player = ap.id  -- Join the shots table with the all_players table on Player and id
    WHERE s.Outcome = 'Goal'  -- Only include records where Outcome = 'Goal'
    GROUP BY ap.name  -- Group by player name from all_players table
    ORDER BY total_goals DESC
    '''

    
    cursor.execute(query)
    results_from_shots = cursor.fetchall()

    # Create a DataFrame from the query results
    df = pd.DataFrame(results_from_shots, columns=['player_name','goals_gt_16', 'goals_lt_16', 'total_goals'])

    # Print the top 10 rows of the DataFrame
    print(df.head(10))

# Call the function and measure execution time
result, execution_time = MySQLQ2_1()


                 player_name goals_gt_16 goals_lt_16  total_goals
0              Kylian-Mbappe         162         720          882
1         Robert-Lewandowski          90         786          876
2                 Harry-Kane         126         594          720
3  Pierre-Emerick-Aubameyang          56         592          648
4              Mohamed-Salah          84         540          624
5              Ciro-Immobile          60         564          624
6          Wissam-Ben-Yedder          18         534          552
7           Lautaro-Martinez          90         444          534
8               Lionel-Messi         185         345          530
9          Cristiano-Ronaldo          72         456          528
Execution time: 0.2940843105316162 seconds
